In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import couponProjectUtils as proj_utils

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

# Use all onehot-keepfirst nominal encoding for coupon type: Carry out & Take away

In [2]:
coupon_type = "CarryAway"

In [3]:
encoding_type = 'onehot-keepfirst' ## use this string to access input file and name output files

os.makedirs(os.path.join('model', encoding_type), exist_ok=True)

In [4]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv(f'data/{coupon_type}_data_{encoding_type}.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE(random_state=42)

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

print("After SMOTE upsampling")
train_vc = y_train.value_counts()
print(f"\tNum training samples: {X_train.shape[0]:,} ({100* train_vc[1]/(train_vc.sum()):.2f}% yes)")
test_vc = y_test.value_counts()
print(f"\tNum testing samples:  {X_test.shape[0]:,} ({100* test_vc[1]/(test_vc.sum()):.2f}% yes)")

Orig: 73.38% Yes
After SMOTE upsampling
	Num training samples: 2,752 (49.38% yes)
	Num testing samples:  688 (52.47% yes)


## Testing RandomForestClassifier with Grid Search Cross-validation


## RandomForestClassifier Parameters

The main parameters to adjust when using these methods:
* n_estimators
* max_features (sqrt is generally best default for classification. I have 55 features, so try values around 8) 

Make sure to include params:
* max_depth=None
* min_samples_split=2 
* oob_score=True (with bootstrapping)

In [5]:
cols_to_view = ['rank_test_score', 'param_max_features', 'param_min_samples_split', 'param_n_estimators', 
#                 'param_min_samples_leaf', 
                'param_max_depth', 'param_max_leaf_nodes',
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

In [6]:
param_grid={'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
            'min_samples_split': [2, 3],
            'n_estimators': [100, 150, 200],
#             'min_samples_leaf': [1, 2, 3, 4, 5, 10],
            'max_depth': [None, 20, 25, 30],
            'max_leaf_nodes': [None, 100, 200]
           }

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

cv_results1 = pd.DataFrame(grid_search1.cv_results_)
best_model = grid_search1.best_estimator_
best_model

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_depth': [None, 20, 25, 30],
                         'max_features': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                          15],
                         'max_leaf_nodes': [None, 100, 200],
                         'min_samples_split': [2, 3],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True, scoring='roc_auc')

RandomForestClassifier(max_depth=25, max_features=6, n_estimators=200,
                       oob_score=True, random_state=42)

In [7]:
print(f"Best model oob score: {best_model.oob_score_}")
y_test_score = best_model.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score):.6f}\n")

Best model oob score: 0.8662790697674418
ROC AUC: TEST 0.930506



In [8]:
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_features,param_min_samples_split,param_n_estimators,param_max_depth,param_max_leaf_nodes,mean_test_score,std_test_score,mean_train_score,std_train_score
416,1,6,2,200,25,None,0.93154362,0.01219399,0.99999051,0.00000576
20,2,6,2,200,None,None,0.93097336,0.01266399,0.99999051,0.00000576
400,3,5,3,150,25,None,0.93090045,0.01181383,0.99994726,0.00002087
401,4,5,3,200,25,None,0.93084375,0.01154512,0.99995436,0.00002150
614,5,6,2,200,30,None,0.93079516,0.01255720,0.99999051,0.00000576
596,6,5,2,200,30,None,0.93078203,0.01128286,0.99999051,0.00000576
203,7,5,3,200,20,None,0.93077148,0.01172984,0.99991705,0.00003308
2,8,5,2,200,None,None,0.93052465,0.01129234,0.99999051,0.00000576
19,9,6,2,150,None,None,0.93051499,0.01245425,0.99999051,0.00000576
613,10,6,2,150,30,None,0.93036644,0.01223552,0.99999051,0.00000576


In [9]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
25,expiration_1d,0.02230745
47,education_Some college - no degree,0.02056778
86,Bar_never,0.01858177
28,gender_Male,0.01788998
106,Restaurant20To50_less1,0.01787797
16,weather_Sunny,0.01773074
94,CarryAway_4~8,0.01710363
98,RestaurantLessThan20_1~3,0.01622899
43,education_Bachelors degree,0.01616814
110,minsToCouponDest_5-14,0.01604369


In [10]:
import pickle

with open(f'model/{encoding_type}/{coupon_type}_GridSearchCV_RFC.pickle', "wb") as f:
    pickle.dump(grid_search1,f)
    
with open(f'model/{encoding_type}/{coupon_type}_best_model_RFC.pickle', "wb") as f:
    pickle.dump(best_model,f)

In [11]:
## save train/test info to file
Xy_tt = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

for k in Xy_tt.keys():
    fname = f'data/{coupon_type}_{k}_{encoding_type}.csv'
    print(f'Saving file: {fname}')
    Xy_tt[k].to_csv(fname, index=False)

Saving file: data/CarryAway_X_train_onehot-keepfirst.csv
Saving file: data/CarryAway_X_test_onehot-keepfirst.csv
Saving file: data/CarryAway_y_train_onehot-keepfirst.csv
Saving file: data/CarryAway_y_test_onehot-keepfirst.csv
